# Notebook: Train Model

* To-Do: Error throwen, wenn es bei den labels ein invalide sequence übersetzung gibt

## Packages

In [ ]:
from load_dataset_folds import load_dataset_folds
from transformers import set_seed
from TASD import train_TASD_model
from ACD import train_ACD_model
from OTE import train_OTE_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

## Parameters

In [ ]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_category" # "aspect_term", "aspect_category", "target_aspect_sentiment_detection"
LLM_SAMPLING = "random"

## Settings

In [ ]:
# Set seeds
torch.device(constants.DEVICE)
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)
set_seed(constants.RANDOM_SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning,
                        module="transformers.optimization")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [ ]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

### Load Model

In [ ]:
if TARGET == "aspect_category":
    results = train_ACD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [ ]:
if TARGET == "aspect_term":
    results = train_OTE_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [ ]:
if TARGET == "target_aspect_sentiment_detection":
    results = train_TASD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

### Save Results

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results

### Remove useless folders

In [ ]:
try:
    shutil.rmtree("outputs")
except:
    pass